In [1]:
%load_ext autoreload
%autoreload 2

import json

import pandas as pd
from pydash import py_

from yiutils.project_utils import find_project_root

# params

In [2]:
proj_root = find_project_root("justfile")
print(proj_root)
data_dir = proj_root / "data"
assert data_dir.exists()

output_dir = proj_root / "output"
assert output_dir.exists()

/lus/lfs1aip1/home/b5a/ik18445.b5a/projects/llm-data-extraction


In [4]:
path_to_mr_pubmed_data = (
    data_dir / "intermediate" / "mr-pubmed-data" / "mr-pubmed-data.json"
)
assert path_to_mr_pubmed_data.exists()

path_to_results_dir = (
    data_dir / "intermediate" / "llm-results" / "isb-ai-116732" / "results"
)
assert path_to_results_dir.exists()

# data loading

In [5]:
with open(path_to_mr_pubmed_data, "r") as f:
    mr_pubmed_data = json.load(f)
    mr_pubmed_df = pd.DataFrame(mr_pubmed_data)
    mr_pubmed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15635 entries, 0 to 15634
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   pmid          15635 non-null  object
 1   ab            15635 non-null  object
 2   pub_date      15635 non-null  object
 3   title         15635 non-null  object
 4   journal_issn  15434 non-null  object
 5   journal       15635 non-null  object
 6   author_affil  15473 non-null  object
dtypes: object(7)
memory usage: 855.2+ KB


In [6]:
json_files = list(path_to_results_dir.glob("*.json"))
print(len(json_files))

json_data = []
for json_file in json_files:
    with open(json_file, "r") as f:
        data = {"data": json.load(f), "filename": str(json_file.name)}
        json_data.append(data)

results_df = pd.concat(
    [
        pd.DataFrame(data["data"]).assign(filename=data["filename"])
        for data in json_data
    ],
).reset_index(drop=True)
tmp_output_file = output_dir / "tmp_results.csv"
results_df.to_csv(tmp_output_file, index=False)
results_df.info()
results_df

70
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   pmid                7000 non-null   object
 1   ab                  7000 non-null   object
 2   pub_date            7000 non-null   object
 3   title               7000 non-null   object
 4   journal_issn        6942 non-null   object
 5   journal             7000 non-null   object
 6   author_affil        6932 non-null   object
 7   metadata            6901 non-null   object
 8   results             7000 non-null   object
 9   resultsinformation  6999 non-null   object
 10  metainformation     498 non-null    object
 11  exposures           99 non-null     object
 12  outcomes            99 non-null     object
 13  methods             104 non-null    object
 14  filename            7000 non-null   object
 15  population          83 non-null     object
dtypes: object(16)
memory 

,pmid,ab,pub_date,title,journal_issn,journal,author_affil,metadata,results,resultsinformation,metainformation,exposures,outcomes,methods,filename,population
0,38025973,Numerous investigations have revealed the inte...,2023-06-30,Investigating the gut microbiota's influence o...,2516-1571,Precision clinical medicine,"Department of Dermatology, Hunan Engineering R...","{'exposures': [{'id': '1', 'trait': 'Gut micro...","[{'exposure': 'Gut microbiota', 'outcome': 'Ps...",{'error': 'No numeric values provided for P-va...,NaN,NaN,NaN,NaN,mr_extract_llama3_sample_array_18.json,NaN
1,38025156,To investigate the causal association between ...,2023-07-10,Causal association between subtypes of osteoar...,2665-9131,Osteoarthritis and cartilage open,"Academic Rheumatology, Clinical Sciences Build...","{'exposures': [{'id': '1', 'trait': 'Osteoarth...","[{'exposure': 'Hip OA', 'outcome': 'Multisite ...","{'error': None, 'explanation': None}",NaN,NaN,NaN,NaN,mr_extract_llama3_sample_array_18.json,NaN
2,38025107,The interactions between fasting insulin level...,2023-11-08,High blood pressure mediated the effect of fas...,2055-2076,Digital health,"Department of Gastroenterology, Liyuan Hospita...","{'exposures': [{'id': '1', 'trait': 'Fasting i...","[{'exposure': 'Fasting insulin levels', 'outco...","{'error': None, 'explanation': None}",NaN,NaN,NaN,NaN,mr_extract_llama3_sample_array_18.json,NaN
3,38023262,The high incidence of breast cancer (BC) promp...,2023-06-24,Prognostic value analysis of cholesterol and c...,2234-943X,Frontiers in oncology,"Department of Breast Surgery, The First Affili...",{},{},{'error': 'Failed Expecting value: line 12 col...,{'error': 'Failed Expecting value: line 12 col...,NaN,NaN,NaN,mr_extract_llama3_sample_array_18.json,NaN
4,38022630,Although numerous observational studies have i...,2023-07-30,Association between genetically predicted rheu...,1664-3224,Frontiers in immunology,"Department of Plastic Surgery, Longyan First A...","{'exposures': [{'id': '1', 'trait': 'Rheumatoi...","[{'exposure': 'Rheumatoid arthritis (RA)', 'ou...","{'error': None, 'explanation': None}",NaN,NaN,NaN,NaN,mr_extract_llama3_sample_array_18.json,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,32628751,Crohn's disease (CD) and ulcerative colitis (U...,2020-01-13,"Cigarette Smoking, Coffee Consumption, Alcohol...",1536-4844,Inflammatory bowel diseases,"Medical School, University of Cyprus, Nicosia,...","{'exposures': [{'id': '1', 'trait': 'Cigarette...","[{'exposure': 'Age of smoking initiation', 'ou...","{'error': None, 'explanation': None}",NaN,NaN,NaN,NaN,mr_extract_llama3_sample_array_51.json,NaN
6996,32628676,To efficiently transform genetic associations ...,2019-06-24,Linking protein to phenotype with Mendelian Ra...,1553-7404,PLoS genetics,"MRC Human Genetics Unit, Institute of Genetics...","{'exposures': [{'id': '1', 'trait': 'SHPS1 (SI...","[{'exposure': 'SHPS1 (SIRPA)', 'outcome': 'sch...","{'error': None, 'explanation': None}",NaN,NaN,NaN,NaN,mr_extract_llama3_sample_array_51.json,NaN
6997,32627641,Background Elevated blood pressure is a major ...,2020-07-07,Genetically Predicted Midlife Blood Pressure a...,2047-9980,Journal of the American Heart Association,Department of Epidemiology and Biostatistics S...,"{'exposures': [{'id': '1', 'trait': 'Midlife b...",[{'exposure': 'Genetically predicted mean arte...,"{'error': None, 'explanation': None}",NaN,NaN,NaN,NaN,mr_extract_llama3_sample_array_51.json,NaN
6998,32610558,Testosterone has historically been linked to s...,2020-06-08,The Potential Effect of Aberrant Testosterone ...,2073-4425,Genes,"Bio-X Institutes, Key Laboratory for the Genet...",{},{},{'error': 'Failed Expecting value: line 48 col...,{'error': 'Failed Expecting value: line 48 col...,NaN,NaN,NaN,mr_extract_llama3_sample_array_51.json,NaN


# basic diagnostics

In [7]:
# num missing
print(results_df["pmid"].isna().sum(), results_df.shape[0])
# num duplicates
print(results_df["pmid"].drop_duplicates().shape[0], results_df["pmid"].shape[0])


0 7000
7000 7000


# results summary

# weird examples

In [8]:
weird_df = results_df[~results_df["exposures"].isna()]
weird_df.info()
weird_df

<class 'pandas.core.frame.DataFrame'>
Index: 99 entries, 88 to 6943
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   pmid                99 non-null     object
 1   ab                  99 non-null     object
 2   pub_date            99 non-null     object
 3   title               99 non-null     object
 4   journal_issn        99 non-null     object
 5   journal             99 non-null     object
 6   author_affil        94 non-null     object
 7   metadata            0 non-null      object
 8   results             99 non-null     object
 9   resultsinformation  99 non-null     object
 10  metainformation     99 non-null     object
 11  exposures           99 non-null     object
 12  outcomes            99 non-null     object
 13  methods             99 non-null     object
 14  filename            99 non-null     object
 15  population          77 non-null     object
dtypes: object(16)
memory usage: 13

,pmid,ab,pub_date,title,journal_issn,journal,author_affil,metadata,results,resultsinformation,metainformation,exposures,outcomes,methods,filename,population
88,37975615,Although a large number of existing studies ha...,2023-10-10,Exploring genetic associations between vitilig...,1600-0625,Experimental dermatology,"Department of Dermatology, The Second Affiliat...",NaN,[],{'error': 'No significant causal associations ...,{'error': 'Some methods do not match those lis...,"[{'id': '1', 'trait': 'Vitiligo', 'category': ...","[{'id': '1', 'trait': 'Depression', 'category'...","[{'id': '1', 'method': 'Inverse-variance weigh...",mr_extract_llama3_sample_array_18.json,NaN
186,37651993,Non-linear Mendelian randomization is an exten...,2022-12-12,Violation of the Constant Genetic Effect Assum...,1423-0062,Human heredity,"MRC Biostatistics Unit, University of Cambridg...",NaN,"[{'exposure': '25-hydroxyvitamin D (25(OH)D)',...","{'error': 'No results provided in abstract', '...",{'error': 'Some methods do not match those lis...,"[{'id': '1', 'trait': '25-hydroxyvitamin D (25...","[{'id': '1', 'trait': 'All-cause mortality', '...","[non-linear Mendelian randomization, residual ...",mr_extract_llama3_sample_array_23.json,[low vitamin D individuals]
203,37390109,Non-linear Mendelian randomization is an exten...,2022-12-26,Relaxing parametric assumptions for non-linear...,1553-7404,PLoS genetics,"MRC Biostatistics Unit, School of Clinical Med...",NaN,"[{'exposure': 'alcohol intake', 'outcome': 'sy...","{'error': 'Some fields are missing', 'explanat...",{'error': 'No information on population is pro...,"[{'id': '1', 'trait': 'Alcohol intake', 'categ...","[{'id': '1', 'trait': 'Systolic blood pressure...","[Non-linear Mendelian randomization, Stratific...",mr_extract_llama3_sample_array_27.json,NaN
268,37316410,Several studies have demonstrated the possible...,2023-01-18,Mendelian Randomisation Study on Association o...,1875-595X,International dental journal,"Department of Oral and Maxillofacial Surgery, ...",NaN,"[{'exposure': 'Lactobacillaceae', 'outcome': '...","{'error': None, 'explanation': None}",{'error': 'No information on population is pro...,"[{'id': '1', 'trait': 'Gut microbiota', 'categ...","[{'id': '1', 'trait': 'Periodontitis', 'catego...","[two-sample mendelian randomization, inverse v...",mr_extract_llama3_sample_array_27.json,[European ancestry]
437,38502664,Multiple Sclerosis (MS) is an immune-related d...,2023-07-10,Association between multiple sclerosis and can...,1932-6203,PloS one,"VIP Inpatient Department, State Key Laboratory...",NaN,"[{'exposure': 'Multiple Sclerosis (MS)', 'outc...","{'error': None, 'explanation': None}",{'error': 'Some methods do not match those lis...,"[{'id': '1', 'trait': 'Multiple Sclerosis (MS)...","[{'id': '1', 'trait': 'cancer', 'category': 'n...","[two-sample MR method, Inverse variance weight...",mr_extract_llama3_sample_array_7.json,[International Multiple Sclerosis Genetics Con...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6551,38453257,"In the most comprehensive analysis to date, th...",2023-11-15,Multi-biobank summary data Mendelian randomisa...,1399-3003,The European respiratory journal,"The MRC Integrative Epidemiology Unit, Univers...",NaN,"[{'exposure': 'CRP-weighted IL-6 signalling', ...","{'error': 'No results provided in abstract', '...",{'error': 'No information on population is pro...,"[{'id': '1', 'trait': 'CRP-weighted IL-6 signa...","[{'id': '1', 'trait': 'PAH risk', 'category': ...",[two-sample mendelian randomization],mr_extract_llama3_sample_array_8.json,[All available PAH GWAS data]
6675,37576893,[This corrects the article DOI: 10.3389/fonc.2...,2023-07-06,Corrigendum: Diseases of the musculoskeletal s...,2234-943X,Frontiers in oncology,"Department of Radiotherapy, The First Affiliat...",NaN,[],"{'error': 'No results provided in abstract', '...","{'error': 'No exposures, outcomes, or methods ...",[],[],[],mr_extract_llama3_sample_array_24.json,[No 